# Setup

In [ ]:
!pip install graphdatascience

In [ ]:
# Import the client
from graphdatascience import GraphDataScience

# Replace with the actual URI, username, and password
AURA_CONNECTION_URI = "neo4j+s://xxxxxxxx.databases.neo4j.io"
AURA_USERNAME = "neo4j"
AURA_PASSWORD = ""

# Configure the client with AuraDS-recommended settings
gds = GraphDataScience(
    AURA_CONNECTION_URI,
    auth=(AURA_USERNAME, AURA_PASSWORD),
    aura_ds=True
)

In [ ]:
# Run the graph generation algorithm and retrieve the corresponding graph object
g, result = gds.beta.graph.generate(
    "example-graph",
    1000000,
    3,
    relationshipDistribution="POWER_LAW"
)

# Print prettified graph stats
print(result)

name                        example-graph
nodes                             1000000
relationships                     2999467
generateMillis                        893
relationshipSeed                     None
averageDegree                         3.0
relationshipDistribution        POWER_LAW
relationshipProperty                   {}
Name: 0, dtype: object


# Run the threads

In [ ]:
# Import to run the long-running algorithm in a thread
import threading
# Import to use the sleep method
import time


# Method to call the label propagation algorithm from a thread
def run_label_prop():
    print("Running label propagation")

    result = gds.labelPropagation.mutate(
        g,
        mutateProperty="communityID"
    )

    print(result)


# Method to get and pretty-print the algorithm progress
def run_list_progress():
    result = gds.beta.listProgress()

    print(result)


# Create a thread for the label propagation algorithm and start it
label_prop_query_thread = threading.Thread(target=run_label_prop)
label_prop_query_thread.start()

# Sleep for a few seconds so the label propagation query has time to get going
print('Sleeping for 5 seconds')
time.sleep(5)

# Check the algorithm progress
run_list_progress()

# Sleep for a few more seconds
print('Sleeping for 10 more seconds')
time.sleep(10)

# Check the algorithm progress again
run_list_progress()

# Block and wait for the algorithm thread to finish
label_prop_query_thread.join()

Running label propagation
Sleeping for 5 seconds
                                  jobId          taskName progress  \
0  02a5afa2-175a-499f-b41c-98fd88b4cc86  LabelPropagation   18.15%   

    progressBar   status         timeStarted elapsedTime  
0  [#~~~~~~~~~]  RUNNING  14:40:02.658000000   5 seconds  
Sleeping for 10 more seconds
                                  jobId          taskName progress  \
0  02a5afa2-175a-499f-b41c-98fd88b4cc86  LabelPropagation   35.98%   

    progressBar   status         timeStarted elapsedTime  
0  [###~~~~~~~]  RUNNING  14:40:02.658000000  16 seconds  
mutateMillis                                                             0
nodePropertiesWritten                                              1000000
ranIterations                                                           10
didConverge                                                          False
communityCount                                                      914596
communityDistribution    {'p9

# Cleanup

In [ ]:
result = gds.graph.drop(g)

print(result)

graphName                                                example-graph
database                                                         neo4j
memoryUsage                                                           
sizeInBytes                                                         -1
nodeCount                                                      1000000
relationshipCount                                              2999467
configuration        {'orientation': 'NATURAL', 'relationshipProper...
density                                                       0.000003
creationTime                       2022-03-29T14:39:25.506835000+00:00
modificationTime                   2022-03-29T14:41:17.239723000+00:00
schema               {'relationships': {'REL': {}}, 'nodes': {'__AL...
Name: 0, dtype: object
